<a href="https://colab.research.google.com/github/lucicom/TensorFlow/blob/main/Hidden_Markov_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hidden Markov Models
The Hidden Markov Model is a finite set of states, each of which is associated with a (generally multidimensional) probability distribution []. Transitions among the states are governed by a set of probabilities called transition probabilities. (http://jedlik.phy.bme.hu/~gerjanos/HMM/node4.html)

A hidden markov model works with probabilities to predict future events or states. In this section we will learn how to create a hidden markov model that can predict the weather.

This section is based on the following TensorFlow tutorial. https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel

### Data
Let's start by discussing the type of data we use when we work with a hidden markov model.

In the previous sections we worked with large datasets of 100's of different entries. For a markov model we are only interested in probability distributions that have to do with states.

We can find these probabilities from large datasets or may already have these values. We'll run through an example in a second that should clear some things up, but let's discuss the components of a markov model.

**States**: In each markov model we have a finite set of states. These states could be something like warm and cold or high and low or even red, green and blue. These states are hidden within the model, which means we do not direcly observe them.

**Observations**: Each state has a particular outcome or observation associated with it based on a probability distribution. An example of this is the following: On a hot day Tim has a 80% chance of being happy and a 20% chance of being sad.

**Transitions**: Each state will have a probability defining the likelyhood of transitioning to a different state. An example is the following: a cold day has a 30% chance of being followed by a hot day and a 70% chance of being follwed by another cold day.

To create a hidden markov model we need.

- States
- Observation Distribution
- Transition Distribution


For our purpose we will assume we already have this information available as we attempt to predict the weather on a given day.

Imports and Setup

In [2]:
!pip install tensorflow_probability --user --upgrade

     |████████████████████████████████| 6.3 MB 5.6 MB/s 
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.1.1
    Uninstalling cloudpickle-1.1.1:
      Successfully uninstalled cloudpickle-1.1.1
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.8.0rc0
    Uninstalling tensorflow-probability-0.8.0rc0:
      Successfully uninstalled tensorflow-probability-0.8.0rc0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.


In [1]:
%tensorflow_version 2.x
import tensorflow_probability as tfp  # We are using a different module from tensorflow this time
import tensorflow as tf
import numpy as np


Here we coded a model for the following:

1. Cold days are encoded by a 0 and hot days are encoded by a 1.
2. The first day in our sequence has an 80% chance of being cold.
3. A cold day has a 30% chance of being followed by a hot day.
4. A hot day has a 20% chance of being followed by a cold day.
5. On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.

**Standard deviation** gives us the range of values above or below the mean


In [2]:
#Here we create probability of distribution model
tfd = tfp.distributions  # making a shortcut for later on
#bracket probability representation - [Cold Day, Hot Day]
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])  # Refer to point 2 above
transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]])  # refer to points 3 and 4 above
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])  # insert standard deviation. Refer to point 5 above
#loc=[mean] scale=[standard deviation]
# the loc argument represents the mean and the scale is the standard devitation"


In [6]:
#Here we use hidden Markov model
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)


The number of steps represents the number of days that we would like to predict information for. In this case we've chosen 7, an entire week.

To get the expected temperatures on each day we can do the following.

In [7]:
mean = model.mean() #Partially defined tensor - This calculates the probablity so that we can run our probability distribution model

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# from within a session to see the value of this tensor

# in the new version of tensorflow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:  
  print(mean.numpy())


[2.9999998 5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]


The model and session combines gives the temperature on 7 days (the reason we input a 7 in the steps portion of the model

Note: temperature is in celsius

- [**Day 1:** 2.9999998 | **Day 2:** 5.9999995 | **Day 3:** 7.4999995 | **Day 4:** 8.25 | **Day 5:** 8.625001 | **Day 6:** 8.812501 | **Day 7:** 8.90625  ]"
